In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import random as rd
import array
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
dataTable = pd.read_csv("/content/drive/My Drive/Colab Notebooks/cardio_train.csv", sep=';').drop('id', 1)

In [ ]:
def ageInYears(dataTable):
    ageInYears = []
    for i in range(len(dataTable)):
        ageInYears.append(int(round(dataTable.iloc[i]['age']/365)))
    return ageInYears

In [ ]:
ageInYears = ageInYears(dataTable)
dataTable['age_years'] = ageInYears
cols = dataTable.columns.tolist()
cols.insert(0, cols.pop(cols.index('age_years')))
dataTable = dataTable.reindex(columns= cols)
del dataTable['age']
dataTable.columns = ['Age_years', 'Gender', 'Height', 'Weight', 'Systolic_bp', 'Diastolic_bp', 'Cholesterol', 'Glucose', 'Smoking', 'Alcohol_intake', 'Physical_activity', 'Cardio_disease' ]

In [ ]:
atributes = dataTable.values[:,:-1]
classes = dataTable.values[:,-1]
xTrain, xTest, yTrain, yTest = train_test_split(atributes, classes, test_size = 0.20)
xTrain, xVal, yTrain, yVal = train_test_split(xTrain, yTrain, test_size = 0.20)

In [ ]:
def normalization(matrix, vector):
    #Mean calculation
    mVector = np.mean(vector, axis=0)
    mMatrix = np.mean(matrix, axis=0)
    
    #Standard deviation
    stdVector = np.std(vector, axis = 0)
    stdMatrix = np.std(matrix, axis = 0)
     
    #Normalization of data
    vector = (vector - mVector)/stdVector
    matrix = (matrix - mMatrix)/stdMatrix
    
    return matrix, vector

In [ ]:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
def predict_logistic_regression_binary(matrix_x, w):
    
    predictions = []
    matrix_x = np.insert(matrix_x, 0, 1, axis =1)
    for i in range(matrix_x.shape[0]):
        predictions.append(round(sigmoide(matrix_x[i].dot(w))))
        
    return predictions

In [ ]:
def stochastic_gradient_descent(matrix_x, vector_y, epochs, alpha):
    matrix_x = np.insert(matrix_x, 0, 1, axis =1)
    w_matrix = np.ones((matrix_x.shape[1],), dtype=float)
    y_predicted = matrix_x.dot(w_matrix)
    data =np.append(matrix_x, np.split(vector_y, matrix_x.shape[0], axis=0), axis=1)
    mse = []

    for epoch in range(epochs):
        erro = 0
        somatorio = 0
        for i in range(matrix_x.shape[0]):
            w_matrix  = w_matrix + alpha * (vector_y[i] - sigmoide(y_predicted[i])) * matrix_x[i]
            somatorio += (vector_y[i] - sigmoide(y_predicted[i])) * matrix_x[i]
        
        mse.append((-1/matrix_x.shape[0]) * somatorio)        
        data = np.random.permutation(data)
        matrix_x = data[: ,0:matrix_x.shape[1]]
        vector_y = data[:,matrix_x.shape[1]]
        
        y_predicted = matrix_x.dot(w_matrix)  
    
    return w_matrix, mse

In [ ]:
def accuracy(predicted, actual):
    total = 0;
    for a,p in zip(actual, predicted):
        if (a == p):
            total +=1;
    return (total/len(actual))*100

In [ ]:
x_train, y_train = normalization(xTrain, yTrain)
x_val, y_val = normalization(xVal, yVal)
x_test, y_test = normalization(xTest, yTest)

In [ ]:
grid_search = np.meshgrid(np.array([0.01, 0.001, 0.0001, 0.00001]), # passo aprendizagem
                          np.array([500, 1000, 1250,1500,2000])) # epocas
scores ={}
for i in range(grid_search[0].shape[0]):
    for j in range(grid_search[1].shape[1]):
        alpha = grid_search[0][i,j]
        lamb = grid_search[1][i,j]
        
        print("passo aprendizagem: ",alpha ,"\t","epocas: " ,lamb)
        
        w, mse = stochastic_gradient_descent(x_train, y_train, lamb, alpha)
        predictions_logistic = predict_logistic_regression_binary(x_val, w)
        acc = accuracy(predictions_logistic, yVal)
        scores[acc] = [alpha, lamb]
        
        print(scores)
        print('accuracy', acc)

best_score = max(scores, key=float)
train_data = np.concatenate((x_train, x_val))
train_test_data = np.concatenate((y_train, y_val))

w, mse = stochastic_gradient_descent(train_data, train_test_data, scores[best_score][1], scores[best_score][0])

predictions_logistic = predict_logistic_regression_binary(x_test, w)
acc = accuracy(predictions_logistic, yTest)
print(acc)

passo aprendizagem:  0.01 	 epocas:  500


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


{57.160714285714285: [0.01, 500]}
accuracy 57.160714285714285
passo aprendizagem:  0.001 	 epocas:  500
{57.160714285714285: [0.001, 500]}
accuracy 57.160714285714285
passo aprendizagem:  0.0001 	 epocas:  500
{57.160714285714285: [0.0001, 500]}
accuracy 57.160714285714285
passo aprendizagem:  1e-05 	 epocas:  500
{57.160714285714285: [0.0001, 500], 57.14285714285714: [1e-05, 500]}
accuracy 57.14285714285714
passo aprendizagem:  0.01 	 epocas:  1000
{57.160714285714285: [0.01, 1000], 57.14285714285714: [1e-05, 500]}
accuracy 57.160714285714285
passo aprendizagem:  0.001 	 epocas:  1000
{57.160714285714285: [0.001, 1000], 57.14285714285714: [1e-05, 500]}
accuracy 57.160714285714285
passo aprendizagem:  0.0001 	 epocas:  1000
{57.160714285714285: [0.001, 1000], 57.14285714285714: [1e-05, 500], 57.169642857142854: [0.0001, 1000]}
accuracy 57.169642857142854
passo aprendizagem:  1e-05 	 epocas:  1000
{57.160714285714285: [1e-05, 1000], 57.14285714285714: [1e-05, 500], 57.169642857142854: [